In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def compute_reverse_histo_norm(histo):
    high_limit = len(histo)

    # First we compute the normalized cumulative histogram
    histo_increasing = np.cumsum(histo)
    histo_norm = (histo_increasing / histo_increasing[-1] * (high_limit-1)).astype(np.uint8)
    
    # Then we compute the reverse histogram
    histo_returned = []
    current_value = -1
    indices_explored = []

    for i, val in enumerate(histo_norm):
        if val != current_value:
            if current_value != -1:
                histo_returned.append(int(np.mean(indices_explored)))
            for k in range(current_value+1, val):
                histo_returned.append(i-1 if i > 0 and (k-current_value)/(val-current_value) <= 0.5 else i)
            indices_explored = [i]
            current_value = val
        else:
            indices_explored.append(i)

    histo_returned.append(int(np.mean(indices_explored)))
    for k in range(current_value+1, high_limit):
        histo_returned.append(high_limit-1)
    
    return histo_returned

In [ ]:
with open("histo_values.json", "r") as f:
    histo_values = json.load(f)

direct_histo = {k: [histo_x[str(i)] if str(i) in histo_x else 0 for i in range(256)] for (k, histo_x) in histo_values.items()}

In [ ]:
reversed_histo = {k: compute_reverse_histo_norm(v) for k, v in direct_histo.items()}

In [ ]:
for k in direct_histo:
    plt.plot(range(len(direct_histo[k])), np.cumsum(direct_histo[k]))
    plt.title(f"{k} cumulative")
    plt.show()
    plt.plot(range(len(reversed_histo[k])), reversed_histo[k])
    plt.title(f"{k} reversed")
    plt.show()

In [ ]:
with open("reversed_histo.json", "w") as f:
    json.dump(reversed_histo, f)